# Visualization notebook

The goal of this notebook is to provide visualization of data using Kepler.gl


In [18]:
from keplergl import KeplerGl
import geopandas as gpd
import pandas as pd

In [19]:
# --- Global variables

# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# --- Paths

# Root path of Fremont Dropbox
import os
import sys
# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [22]:
def to_gdf(path):
    """
    Parameters: 
        path: path of the file to read as a geodataframe
        
    return:
        a GeoDataFrame (with Geopandas) corresponding to the file path
    """
    gdf = gpd.GeoDataFrame.from_file(path)
    gdf = gdf.to_crs('epsg:4326')
    return gdf

## 1. Network data

## Aimsun map

In [6]:
aimsun_path_regular = dbx + '/Private Structured data collection/Aimsun/Inputs/'
aimsun_path_complex = dbx + '/Private Structured data collection/Aimsun/Inputs_complex/'

aimsun_path = aimsun_path_complex

detectors = to_gdf(aimsun_path +'detectors.shp')
meterings = to_gdf(aimsun_path +'meterings.shp')
# centroids = to_gdf(aimsun_path +'centroids.shp')
# centroid_connections = to_gdf(aimsun_path +'centroid_connections.shp')
nodes = to_gdf(aimsun_path +'nodes.shp')
# polygons = to_gdf(aimsun_path +'polygons.shp')
sections = to_gdf(aimsun_path +'sections.shp')
# sectionsGeo = to_gdf(aimsun_path +'sectionsGeo.shp')
# turnings = to_gdf(aimsun_path +'turnings.shp')

In [13]:

fremont_map = KeplerGl(height=600)
# fremont_map.add_data(data = detectors, name="Detectors")
# fremont_map.add_data(data = meterings, name="meterings")
fremont_map.add_data(data = nodes, name="nodes")
# fremont_map.add_data(data = polygons, name="polygons")
fremont_map.add_data(data = sections, name="sections")
# fremont_map.add_data(data = sectionsGeo, name="sectionsGeo")
# fremont_map.add_data(data = turnings, name="turnings")
# fremont_map.add_data(data = centroids, name="centroids")
# fremont_map.add_data(data = centroid_connections, name="centroid_connections")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'nodes': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 2…

In [8]:
print(sections['rd_type'].count())
print(sections['rd_type'].unique())
print("175: " + str(sections[sections['rd_type'] == 175.0]['id'].count()))
print("177: " + str(sections[sections['rd_type'] == 177.0]['id'].count()))
print("179: " + str(sections[sections['rd_type'] == 179.0]['id'].count()))
print("180: " + str(sections[sections['rd_type'] == 180.0]['id'].count()))
print("182: " + str(sections[sections['rd_type'] == 182.0]['id'].count()))
print("184: " + str(sections[sections['rd_type'] == 184.0]['id'].count()))
print("185: " + str(sections[sections['rd_type'] == 185.0]['id'].count()))

7321
[175. 179. 182. 185. 180. 184. 177.]
175: 150
177: 439
179: 3369
180: 498
182: 286
184: 62
185: 2517


In [11]:
print(nodes['nodetype'].unique())

print("0: " + str(nodes[nodes['nodetype'] == 0.0]['id'].count()))
print("1: " + str(nodes[nodes['nodetype'] == 1.0]['id'].count()))
print("2: " + str(nodes[nodes['nodetype'] == 2.0]['id'].count()))
print("3: " + str(nodes[nodes['nodetype'] == 3.0]['id'].count()))

[3. 0. 2. 1.]
0: 774
1: 1632
2: 44
3: 190


## Traffic signals and stop signs

In [29]:
def to_gdf_csv(path):
# https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html#from-wkt-format
    df = pd.read_csv(path)
    gdf = gpd.GeoDataFrame(
        df, crs='epsg:4326', geometry=gpd.points_from_xy(df.x, df.y))
    return gdf

In [31]:
network_infra_path = data_path + "/Manual-made dataset (do not touch)/Network/Infrastructure/"

stop_signs = to_gdf_csv(network_infra_path + "Stop signs location/Stop_Signs.csv")
traffic_lights = to_gdf_csv(network_infra_path + "Traffic lights location/Traffic_Lights.csv")

In [32]:
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = stop_signs, name="Stop signs")
fremont_map.add_data(data = traffic_lights, name="Traffic lights")
fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Stop signs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, …

In [33]:
print(stop_signs.describe())

       __OBJECTID           x           y
count   313.00000  313.000000  313.000000
mean    222.99361 -121.927146   37.520037
std      98.13302    0.010298    0.015053
min      33.00000 -121.952584   37.489385
25%     150.00000 -121.932672   37.507222
50%     228.00000 -121.925565   37.520876
75%     306.00000 -121.919649   37.531509
max     384.00000 -121.908920   37.551676


In [34]:
path_taz = data_path + "/Data processing/Auxiliary files/Demand/OD demand/TAZ"
internal_taz = to_gdf(path_taz + "/Internal_TAZ.shp")
external_taz = to_gdf(path_taz + "/External_TAZ.shp")

In [35]:
fremont_map = KeplerGl(height=600)
fremont_map.add_data(data = internal_taz, name="Internal TAZs")
fremont_map.add_data(data = external_taz, name="External TAZs")
Data\ processing/Temporary\ exports\ to\ be\ copied\ to\ processed\ data/Demand/OD\ demand fremont_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Internal TAZs': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1…

In [37]:
od_demand_path = data_path + "/Data processing/Temporary exports to be copied to processed data/Demand/OD demand/" 
internal_od = pd.read_csv(od_demand_path + "Internal OD grouped by timestamp.csv")
external_od = pd.read_csv(od_demand_path + "External OD grouped by timestamp.csv")

In [47]:
internal_od_4_pm = internal_od[internal_od["dt_15"]=="16:00"]
external_od_4_pm = external_od[external_od["dt_15"]=="16:0"]

print(internal_od_4_pm.counts.sum())
print(external_od_4_pm.counts.sum())


291
1268


In [43]:
print(external_od.head())

   Unnamed: 0 CentroidID_O CentroidID_D dt_15  counts
0          20       ext_13        ext_4  14:0      27
1          21       ext_13       ext_20  14:0     354
2          22       ext_13       ext_23  14:0       0
3          23       ext_13       ext_21  14:0     188
4          24        ext_4       ext_13  14:0      14
